In [5]:
from sqlalchemy import create_engine
conn=create_engine("mysql+mysqldb://root@localhost/test")

In [6]:
type(conn)

sqlalchemy.engine.base.Engine

In [7]:
import pandas as pd
df = pd.read_excel('data.xlsx')
df.to_sql(con=conn,name='base_teste', if_exists='replace')

### 1 - Qual a conversão de vendas no mês de abr/2020?(Para a conversão considere os negócios com status "Ganho"/total de negócios)													


In [8]:
myQuery = '''SELECT COUNT(Status) AS 'Total Conversão'
            FROM base_teste
            WHERE Status='Ganho' and `Data Fechamento` BETWEEN '2020-04-01' and '2020-04-30';'''
pd.read_sql_query(myQuery, conn)

,Total Conversão
0,178


### 2 - Em qual mês de 2019 houve o maior número de negócios perdidos?													
													

In [9]:
myQuery = '''
                SELECT COUNT(Status) AS Total, MONTH(`Data Fechamento`) AS Mês
                FROM base_teste
                WHERE Status='Perdido' and `Data Fechamento` BETWEEN '2019-01-01' and '2019-12-31'
                GROUP BY MONTH(`Data Fechamento`)
                ORDER BY COUNT(Status) DESC
                LIMIT 1;
            '''
pd.read_sql_query(myQuery, conn)

,Total,Mês
0,1903,4


### 3 - Crie um gráfico com o total de vendas(negócios com status "Ganho") mensal dos anos 2019 e 2020, com uma linha de tendência linear.													


In [10]:
myQuery= '''
SELECT COUNT(Status) AS Total,MONTH(`Data Fechamento`) AS Mês, YEAR(`Data Fechamento`) AS Ano
FROM base_teste
WHERE Status='Ganho' and `Data Fechamento` BETWEEN '2019-01-01' and '2020-12-31'
GROUP BY MONTH(`Data Fechamento`)
ORDER BY `Data Fechamento`
'''
pd.read_sql_query(myQuery, conn)

,Total,Mês,Ano
0,625,2,2019
1,467,3,2019
2,426,4,2019
3,281,6,2019
4,379,7,2019
5,391,8,2019
6,378,9,2019
7,381,10,2019
8,415,11,2019
9,182,12,2019


### 4 - Em qual dia da semana ocorreu o maior número de vendas no mês de fev/2020?	

In [11]:
myQuery = '''
            SELECT COUNT(Status) AS totalVendas, DAYOFWEEK(`Data Fechamento`) AS DayofWeek
            FROM base_teste
            WHERE Status='Ganho' and `Data Fechamento` BETWEEN '2020-02-01' and '2020-02-28'
            GROUP BY DAY(`Data Fechamento`)
            ORDER BY totalvendas DESC
            LIMIT 1;'''
pd.read_sql_query(myQuery, conn)

,totalVendas,DayofWeek
0,66,6


### 5 - Qual o tempo médio em dias do fechamento dos negócios em jan/2020?			

In [12]:
myQuery = '''
            SELECT COUNT(DISTINCT(`Data Fechamento`))/COUNT(Status) AS tempoMedio
            FROM base_teste
            WHERE Status='Ganho' and `Data Fechamento` BETWEEN '2020-01-01' and '2020-01-31';'''
pd.read_sql_query(myQuery, conn)

,tempoMedio
0,0.0502


### 6 - Qual proprietário possui a maior média de atividades concluídas em 2019?(Considere proprietários com mais de 40 negócios)													


In [13]:
myQuery = '''
            SELECT `Proprietário`, COUNT(Status) as Negocios, AVG(`Atividades Concluídas`) as Media
            FROM base_teste
            WHERE `Data Fechamento` BETWEEN '2019-01-01' and '2019-12-31'
            GROUP BY `Proprietário`
            HAVING Negocios > 40
            ORDER BY Media DESC
            LIMIT 3;'''
pd.read_sql_query(myQuery, conn)

,Proprietário,Negocios,Media
0,Rafael Fontana,53,10.5472
1,Maic Freitas,60,8.5167
2,Michel Vidal,49,8.0204


### 7 - Crie uma análise estatística da base de dados no ano de 2020 com abertura por mês (Utilize médias, correlação, quartil e mais funções que julgar relevantes)										

In [14]:
# Histograma
myQuery = '''
     SELECT floor(`Atividades Concluídas`/10)*10 as faixa_atividades, count(*) as freq
     FROM base_teste
     group by 1
     order by 1;'''
pd.read_sql_query(myQuery, conn)

,faixa_atividades,freq
0,0.0,38623
1,10.0,2221
2,20.0,207
3,30.0,39
4,40.0,13
5,50.0,1
6,60.0,2
7,120.0,1


### 8 - Analisando o evolutivo mensal dos negócios do período entre jan/19 e jul/19, qual o índice de correlação apresentado entre o total de negócios e o total de negócios perdidos? A partir desse índice é possível afirmar que há uma relação entre esses dois indicadores? Justifique.		

In [15]:
# Correlação de Pearson
myQuery='''
SELECT 
        (
            sum(x*y)-   (
                        (sum(x)*sum(y))/count(*)
                        )
        )
        /
        (
            (
                sqrt(sum(x*x)-((sum(x)*sum(x))/count(*)))                
            )
            *sqrt(sum(y*y)-((sum(y)*sum(y))/count(*)))
        ) AS pearson

FROM(
SELECT 
    SUM(IF(Status='Ganho',Total,0)) as x,
    SUM(IF(Status='Perdido',Total,0)) as y,
    Mês
FROM(
SELECT 
    COUNT(Status) as Total,
    MONTH(`Data Fechamento`) as Mês,
    Status
FROM base_teste
WHERE `Data Fechamento` BETWEEN '2019-01-01' and '2019-07-31'
GROUP BY MONTH(`Data Fechamento`),Status
) temp
GROUP BY Mês
) temp2;
'''
pd.read_sql_query(myQuery, conn)

,pearson
0,-0.094604


In [16]:
import requests
response = requests.get(
'https://api.pontomais.com.br/external_api/v1/employees/11/team_changes?attributes=id,team,date&count=true&page=1&per_page=10&sort_direction=asc&sort_property=first_name',
headers={'Authorization': '$2a$10$FgvbKMpP5PpHG/PBrb8KLOZZLqMIoPCYakT.04lyDJc/7SIKs2sxu'})
print(response)

<Response [404]>
